# Créer un logiciel grâce à Electron

On commence simplement par créer le projet et entrer dedans

```bash
mkdir new-project && cd new-project
```

Puis on initialise la création de projet avec npm

```bash
npm init
```

Voici un exemple de package :

```json
{
  "name": "new-project",
  "version": "0.1.0",
  "description": "The aim of this software is to facilitate the exchange of data between users",
  "main": "main.js",
  "author": "haDock404",
  "license": "MIT"
}
```

Et enfin on installe ```Electron```

```bash
npm install electron --save-dev
```

On crée le frontend pour l'utilisation de Reac

```bash
npx create-react-app frontend --template cra-template
cd frontend
```

Installation manuel des vitals je ne sais pas pourquoi ça ne marche plus automatiquement 
```bash
npm i web-vitals --save-dev
```

On ajoute ceci dans le ````package.json```` de React

```json
"name": "frontend",
"version": "0.1.0",
"homepage": ".", ///<-- permet d'utiliser la racine dans la manipulation du .html
```

On ajoute ceci dans le ```package.json``` d'Electron

```json
"scripts": {
    "start": "electron .",
    "build-react": "cd frontend && npm run build",
    "start-react": "cd frontend && npm start",
    "test": "echo \"Error: no test specified\" && exit 1"
}
```

On construit l'application React :

```bash
cd frontend
npm run build
cd ..
```

On crée un fichier ```main.js``` et ```preload.js```

```js
//main.js

const { app, BrowserWindow } = require('electron');
const path = require('path');

let mainWindow;

const createWindow = () => {
    mainWindow = new BrowserWindow({
        width: 800,
        height: 600,
        webPreferences: {
            preload: path.join(__dirname, 'preload.js'),
        },
    });

    // Chargez l'application React
    const startUrl = `file://${path.join(__dirname, 'frontend', 'build', 'index.html')}`;
    console.log('Loading URL:', startUrl);

    mainWindow.loadURL(startUrl);

    // Ouvrir les DevTools
    //mainWindow.webContents.openDevTools();

    // Gestion des événements de chargement
    mainWindow.webContents.on('did-finish-load', () => {
        console.log('Page loaded successfully');
    });

    mainWindow.webContents.on('did-fail-load', (event, errorCode, errorDescription) => {
        console.log('Failed to load:', errorDescription);
    });

    mainWindow.webContents.on('console-message', (level, message) => {
        console.log(`Console message [${level}]: ${message}`);
    });
};

app.whenReady().then(createWindow);

app.on('window-all-closed', () => {
    if (process.platform !== 'darwin') {
        app.quit();
    }
});

app.on('activate', () => {
    if (BrowserWindow.getAllWindows().length === 0) {
        createWindow();
    }
});
```

A la racine lancer l'application :

```bash
npm run start
```

## Utilisation de python dans une application

#### Réaliser une structure de projet identique :

````
app/
    - main.js
    - renderer.js
    - index.html

python/
    - app.py
    - requirements.txt

python_env/
    - ...

package.json
.env
README.md
```

#### Créer un environnement Conda :

```bash
conda create -p ./python_env python=3.10.9
conda activate ./python_env
pip install -r python/requirements.txt
```

#### Utilisation de python dans l'application Electron

```js
//main.js

const { app, BrowserWindow, ipcMain } = require('electron');
const path = require('path');
const { spawn } = require('child_process');

let mainWindow;

app.on('ready', () => {
  mainWindow = new BrowserWindow({
    width: 800,
    height: 600,
    webPreferences: {
      nodeIntegration: true,
      contextIsolation: false,
    },
  });

  mainWindow.loadFile('app/index.html');
});

ipcMain.on('run-python', (event, args) => {
  // Chemin vers le Python
  const pythonPath = path.join(__dirname, '..', 'python_env', 'bin', 'python');
  const pythonScript = path.join(__dirname, '..', 'python', 'app.py');

  const pythonProcess = spawn(pythonPath, [pythonScript, args]);

  pythonProcess.stdout.on('data', (data) => {
    event.reply('python-output', data.toString());
  });

  pythonProcess.stderr.on('data', (data) => {
    console.error(`Python Error: ${data}`);
    event.reply('python-error', data.toString());
  });
});

app.on('window-all-closed', () => {
  app.quit();
});
```

#### HTML de base

```html
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Application Electron avec Python</title>
</head>
<body>
  <h1>Calcul avec NumPy</h1>
  <input id="input" type="text" placeholder="Entrez un nombre">
  <button id="calculate">Calculer</button>
  <p id="result">Résultat : </p>

  <script src="renderer.js"></script>
</body>
</html>
```

#### Script de l'interface 

```js
const { ipcRenderer } = require('electron');

document.getElementById('calculate').addEventListener('click', () => {
  const input = document.getElementById('input').value;
  ipcRenderer.send('run-python', input);
});

ipcRenderer.on('python-output', (event, data) => {
  document.getElementById('result').textContent = `Résultat : ${data}`;
});

ipcRenderer.on('python-error', (event, data) => {
  document.getElementById('result').textContent = `Erreur : ${data}`;
});
```

#### Code du script Pyhton

```python
import sys
import numpy as np

def main():
    if len(sys.argv) != 2:
        print("Usage: python app.py <number>")
        return

    try:
        num = float(sys.argv[1])
        result = np.square(num)
        print(result)
    except ValueError:
        print("Veuillez entrer un nombre valide.")

if __name__ == "__main__":
    main()
```

#### Installation de l'empacteur pour créer l'application

```bash
npm install electron-packager --save-dev
```

#### Ajouter le script d'empactage pour mac

```json
"scripts": {
  "package": "electron-packager . ElectronPythonApp --platform=darwin --arch=x64 --out=release-build --overwrite --extra-resource=python_env --extra-resource=python"
}
```

#### Lancement de l'empactage
```bash
npm run package
```

#### Création d'un binaire multiplateforme

```bash
npm install electron-builder --save-dev
```

#### Script d'empactage

```json
"scripts": {
  "build:mac": "electron-builder --mac",
  "build:win": "electron-builder --win",
  "build:linux": "electron-builder --linux"
}
```

#### Lancement de la commande pour les différents OS

```bash
npm run build:mac
npm run build:win
npm run build:linux
```
